![Image](./resources/cropped-SummerWorkshop_Header.png)

<h1 align="center">Population Coding: Part 1</h1> 
<h2 align="center"> Day 2, Morning Session</h2> 

<h3 align="center">Tuesday, August 22, 2023</h3> 

<br>
<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">


<h2> The Questions </h2>
    
<p> The first week of the course is devoted to question focused workshops and addresing those questions with datasets and computational methods that we will present.  Each day will be devoted to its own question:  
    

<li> Day 1:  How do single neurons encode information about the external world?  
<b><li> Day 2:  How do populations of neurons encode information about the external world? </b>
<li> Day 3:  How do cell type and circuit architecture contribute to physiological function?
<li> Day 4:  What is a behavioral state and how can we define it? 
<li> Day 5:  How does network structure lead to network dynamics?

## Introduction

In yesterday's lesson, we learned how to compute tuning curves for individual neurons, to better understand how they encode information about the environment (specifically the direction of a drifting grating stimulus). Today, we will expand our focus to **how information is encoded in *neural populations***. Understanding how populations of neurons create models of the world that are used to guide behavior is a central goal of systems neuroscience. In this tutorial, we will learn techniques for computing the information that can be read out of population activity *in principle*, which can be used to contrain hypotheses about how the brain uses this information in practice.

We will also introduce a **new dataset**, the Allen Brain Observatory Visual Coding Neuropixels. This dataset, which was released in 2019, is similar to the 2P dataset covered yesterday, but includes electrophysiological recordings of spike times instead of calcium-dependent fluorescence traces. This dataset is ideal for studying population coding because it includes simultaneous recordings from many brain regions in mice passively viewing a wide range of visual stimuli.

In addition, we will build upon yesterday's lesson by using `scikit-learn` to perform **classification** instead of regression. The techniques for building design matrices and fitting model will be re-used in this tutorial.  

## Dataset overview

The **Allen Brain Observatory Visual Coding Neuropixels** pipeline was designed with a number of goals in mind:
- Conduct *in vivo* electrophysiology experiments in a higher throughput, more standardized manner than was possible previously
- Reproduce the same stimuli from the 2P observatory, in order to allow a direct comparison of imaging and electrophysiology data
- Record activity simultaneously in multiple brain regions, including sub-cortical areas such as hippocampus and thalamus

The rigs for this pipeline were designed to as similar as possible to the 2P rigs from the mouse's perspective; we've just replaced the imaging objective (left) with an array of six Neuropixels probes (right):

![Ophys vs. ephys rig comparison](./resources/ophys_vs_ephys_rig_comparison.png)

Electrophysiology (or "ephys") has a number of important differences to 2P imaging that must be considered when deciding which type of data is best suited for your scientific question:
- **Temporal resolution:** Ephys samples extracellular voltages around 30,000 times per second, making it possible to detect voltage deflections that correspond to action potentials ("spikes"). 2P imaging samples calcium-dependent fluorescence changes at around 30 times per second, making it difficult to resolve single spikes.
- **Spatial configuration:** Neuropixels probes sample voltages along a straight line that can extend into deep structures, all the way down to the bottom of the brain. 2P imaging is limited to sampling a plane up to a few hundred microns below the surface, unless a large lens is implanted in the brain.
- **Cell type specificity:** Ephys can label one genetically defined cell type per experiment through optotagging, but most of the recorded cells have unknown cell type identities. By expressing calcium indicators in a genetically defined sub-population, 2P imaging ensures that *all* the recorded cells are from the same class.
- **Chronic vs. acute:** Current techniques greatly limit the number of neurons that can be recorded chronically over many days in a mouse brain, relative to preparations in which electrodes are inserted and removed each day. 2P imaging, on the other hand, makes it possible to return to the same field of view over many weeks.

Here is a side-by-side comparison of 2P imaging (left) and Neuropixels (right) data from 50 simultaneously recorded neurons in Visual Cortex. Note that the 2P data consists of continuous dF/F traces that vary in size over several orders of magnitude, while the Neuropixels data consists of discrete spike times:

![Ophys vs. ephys data comparison](./resources/ophys_vs_ephys_data_comparison.png)

For this tutorial, we've chosen to use the Neuropixels dataset, but similar techniques could be applied to 2P imaging data as well.

## Tutorial goals

We will first introduce the Allen Brain Observatory Neuropixels dataset, and use it to read out information about stimulus identity from neural populations. We will focus on activity in primary visual cortex (Allen CCF acronym: `VISp`) and drifting gratings stimuli, but the methods will generalize to other areas and stimulus types.

We will explore how populations of neurons encode information by attempting to use a classifier to *decode* drifting grating orientation from simultaneously recorded spike trains. This involves training different classifiers to identify the stimulus parameters on held-out trials. We will use two types of classifiers, Support Vector Machines and Random Forests, but almost any [`scikit-learn` classifier](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html) could be substituted instead.

Let's start by importing some of the packages we'll need:

In [ ]:
import numpy as np    # used for creating and manipulating arrays

import matplotlib.pyplot as plt  # used for plotting data
%matplotlib inline

And setting the path to the data, depending on where we're running the code:

In [ ]:
import platform
import os

platstring = platform.platform()
if ('Darwin' in platstring) or ('macOS' in platstring):
    # macOS 
    data_root = "/Volumes/Brain2023/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn' in platstring):
    # then on Code Ocean
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2023/"

### Exploring the Neuropixels Visual Coding dataset

The Allen Brain Observatory Neuropixels Visual Coding dataset consists of 58 passive viewing sessions that use many of the same stimuli as the 2P Visual Coding experiments we learned about yesterday.

We will access the data via the AllenSDK. The gateway to the data is through an `EcephysProjectCache` object. If you're analyzing data on a local machine, this object will keep track of which files you have downloaded, and retrieve additional files as needed. In this case, we're doing our analysis in the cloud, and the public S3 bucket containing all of the data files is already mounted. So we just need to make sure the `EcephysProjectCache` object is aware of the location where the data is stored:

In [ ]:
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

manifest_path = os.path.join(data_root,'allen-brain-observatory/visual-coding-neuropixels/ecephys-cache/manifest.json') 

cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

We can use the `cache` object to retrieve a table containing information about all of the experimental sessions:

In [ ]:
sessions = cache.get_session_table()

sessions.head()

Each session is indexed by a unique `id`, and has other columns with information about the subject, session type, etc.

We can examine at a single row to see what information is available about each session:

In [ ]:
sessions.iloc[0]

We have information about the subject (e.g. age, sex, genotype), as well as the total number of units recorded, and a list of structures they were recorded from. We use the term "units" instead of "neurons" because each unit may actually include spikes from multiple nearby neurons. The units that are returned by default are pre-filtered to remove those with high levels of contamination (based on the `isi_violations` metric), but they are  not guaranteed to all correspond to perfectly isolated neurons.

We can filter this table to find, for example, the sessions carried out in Sst-IRES-Cre mice, which are used to optotag somatostatin-positive interneurons.  We'll use the Pandas [`map`](https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html) method to apply the same function to every element in a column. In this case, the method must return either `True` or `False` to indicate whether a particular row is of interest:

In [ ]:
def check_genotype(genotype):
    return 'Sst' in genotype

sessions_Sst = sessions[sessions['full_genotype'].map(check_genotype)]

len(sessions_Sst)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    
<b>Task:</b> How would you use this table to find all of the sessions that include recordings from a particular brain region (e.g. <tt>VISp</tt>)?
</div>

In [ ]:
# enter code here

# hint: create a function called `check_for_region` similar to `check_genotype`

One of the most important things to consider about each session is the `session_type`, of which there are two in the Neuropixels Visual Coding dataset. We can use the Pandas [`unique`](https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html) method to find the unique values in the `session_type` column:

In [ ]:
sessions.session_type.unique()

The two session types differ in the visual stimuli that were used. `brain_observatory_1.1` sessions include many stimuli that are identical to the ones shown in the 2P Visual Coding experiments, to facilitate direct comparisons between these two data modalities. `functional_connectivity` sessions use similar stimuli but with fewer conditions and more repeats, to facilitate analysis of cross-area interactions. 

A timeline of each session type is available in the Neuropixels Cheat Sheet PDF, and is also reproduced here:

![Diagram of stimuli for each session type](./resources/neuropixels_visual_stimuli-2.png)

For this tutorial, we are going to use the "Brain Observatory 1.1" sessions, which include the identical drifting grating stimulus as the 2P experiments.

Every unit that was recorded has some pre-computed functional metrics based on its response to each stimulus type, which can be accessed as a `DataFrame` via the `cache` object:

In [ ]:
unit_metrics = cache.get_unit_analysis_metrics_by_session_type('brain_observatory_1.1')

len(unit_metrics)

To find the metrics computed based on responses to the drifting gratings (`dg`) stimulus, we can search for all the metric names with `dg` appended:

In [ ]:
dg_metrics = [column_name for column_name in unit_metrics.columns if 'dg' in column_name]
dg_metrics

There are 14 different metrics for the drifting gratings (`dg`) stimulus type, which have been computed for all units in this dataset. Later, we'll use the `g_dsi_dg` metric to find units in V1 that are most selective for grating direction. `g_dsi` is the acronym for "global direction selectivity index," which is a measure of how much a neuron is tuned to a single grating direction (this is related to how informative its spikes will be about the particular grating being presented).

Let's filter the `unit_metrics` `DataFrame` to select a session that has a reasonably large number of units in V1:

In [ ]:
# get the IDs for all sessions with units in VISp
session_ids = unit_metrics[unit_metrics.ecephys_structure_acronym == 'VISp'].ecephys_session_id.values 

# find the unique session IDs, and count how many there are of each
unique_sessions, counts = np.unique(session_ids, return_counts=True) 
order = np.argsort(counts)

print('Unique session IDs:')
print(unique_sessions[order])
print(' ')
print('Count of units in V1:')
print(counts[order])

Let's analyze session `798911424`, which has 94 V1 units. Note that these unit counts are generated *after* default quality control metrics are applied, so we can be reasonably confident that these units were recorded throughout the whole session and are free of contamination. For more information about unit quality metrics and how to use them, please see [this tutorial](https://allensdk.readthedocs.io/en/latest/_static/examples/nb/ecephys_quality_metrics.html).

### Loading data for one session

We can load the data for this session based on the unique session ID:

In [ ]:
session_id = 798911424

session = cache.get_session_data(session_id)

The `session` object we just created will serve as the interface to all of the data for this experiment, including the spike times, stimulus info, and behavioral data (e.g. running speed).

First, we'll load a table of units for this session. Note that this step can sometimes be slow, since all of the spike times need to be loaded into memory.

In [ ]:
units = session.units # can be slow -- needs to load all spike times

units.head()

The units table contains important information about each unit that was recorded, including its spike sorting quality metrics, its 3D position in the Allen Common Coordinate Framework, and the structure in which it's located. 

Here is a brief summary of the columns in this table:

|column name | description |
|------------|-------------|
|`unit_id` | the identifier for this unit assigned by the AllenSDK (unique across the entire dataset)|
|`waveform_PT_ratio`| peak-to-trough ratio of the average spike waveform |
|`waveform_amplitude`| amplitude (in microvolts) of the average spike waveform |
|`amplitude_cutoff`| a measure of the approximate fraction of spikes missing from this unit (default threshold = 0.1)|
|`cluster_id` | the identifier for this unit assigned by the spike sorting algorithm (unique within each probe)|
|`cumulative_drift` | the integrated distance (in microns) that the unit drifted across the whole session |
|`d_prime` | a measure of how separable this unit's waveforms are from its neighbors' |
|`firing_rate` | mean spike rate across the whole session |
|`isi_violations` | a measure of this unit's level of contamination (default threshold = 0.5) |
|`isolation_distance` | a measure of how separable this unit's waveforms are from its neighbors' (higher is better) |
|`L_ratio` | a measure of how separable this unit's waveforms are from its neighbors' (lower is better) |
|`local_index` | the index of this unit within the probe it was recorded with |
|`max_drift` | the maximum distance (in microns) the unit drifted across the whole session |
|`nn_hit_rate` | a measure of this unit's level of contamination |
|`nn_miss_rate` | a measure of the fraction of spike missing from this unit |
|`peak_channel_id` | the identifier for this unit's peak channel (can be used as an index into the `session.channels` table |
|`presence_ratio` | the fraction of the session over which this unit had spikes detected (default threshold = 0.9) |
|`waveform_recovery_slope` | slope of the waveform between the trough and the peak |
|`waveform_repolarization_slope` | slope of the waveform back to 0 after the peak |
|`silhouette_score` | a measure of this unit's level of contamination |
|`snr` | the ratio of the waveform amplitude relative to the background noise on the peak channel |
|`waveform_spread` | distance the waveform extends above and below the peak channel |
|`waveform_velocity_above` | speed of waveform propagation above the peak channel |
|`waveform_velocity_below` | speed of waveform propagation below the peak channel |
|`waveform_duration` | time between the waveform peak and trough |
|`filtering` | filter properties of the probe used to record this unit |
|`probe_channel_number` | local index of this unit's peak channel |
|`probe_horizontal_position` | horizontal position of this unit on the probe |
|`probe_id` | identifier of the probe used to record this unit |
|`probe_vertical_position` | vertical position of this unit on the probe |
|`structure_acronym` | CCF region where this unit is located |
|`ecephys_structure_id` | CCF structure ID where this unit is located |
|`ecephys_structure_acronym` | alias for `structure_acronym` |
|`anterior_posterior_ccf_coordinate` | CCF coordinate along the A/P axis |
|`dorsal_ventral_ccf_coordinate` | CCF coordinate along the D/V axis |
|`left_right_ccf_coordinate` | CCF coordinate along the L/R axis |
|`probe_description` | name of the probe used to record this unit |
|`location` | not used |
|`probe_sampling_rate` | spike band sampling rate of the probe used to record this unit |
|`probe_lfp_sampling_rate` | LFP band sampling rate of the probe used to record this unit |
|`probe_has_lfp_data` | `True` if LFP data was recorded on the same probe |


The primary data we'd like to analyze for unit is its spike train. Spike times for each unit are contained in the `spike_times` dictionary of the `session` object, and are accessed by `unit_id`:

In [ ]:
session.spike_times[units.index.values[0]]

We can plot a raster of spike times by looping through all the units in this session:

In [ ]:
times = []
unit_indices = []

for unit_idx, unit_id in enumerate(units.index.values):
    spike_times = session.spike_times[unit_id]
    in_range = np.where(spike_times < 10)[0]
    times.append(spike_times[in_range])
    unit_indices.append([unit_idx] * len(in_range))
    
plt.figure(figsize=(15,5))
plt.scatter(np.concatenate(times),
            np.concatenate(unit_indices),
            s=1,
            c=np.concatenate(unit_indices),
            cmap='magma')
plt.xlabel("Time (s)")
plt.ylabel("Unit index")

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    
<b>Task:</b> Plot a spike raster from 1000 to 1010 seconds, for just the cells in primary visual cortex.
</div>

In [ ]:
# code goes here

# hint: copy and paste the code from the previous cell, and edit as needed

Next, we'll load a table of containing information about all of the stimuli that were shown. 

**Note:** Before we do this, we need to replace one of the functions in the AllenSDK with a function that returns its input unchanged. This is due to a discrepancy between the NWB files contained in the AWS bucket and what is expected by the latest version of the AllenSDK. This patch is not normally be needed, but it's helpful to understand in case you encounter this issue in the future.

In [ ]:
from allensdk.brain_observatory.ecephys.stimulus_table import naming_utilities

def passthrough_function(input):
    return input

naming_utilities.standardize_movie_numbers = passthrough_function
session.naming_utilities = naming_utilities # this is needed to circumvent a bug in the AllenSDK

Now we can load the stimulus presentations table without errors:

In [ ]:
stimulus_presentations = session.stimulus_presentations

stimulus_presentations.head()

`stimulus_presentations` is a `DataFrame` containing one row for each presentation or "trial." We can see all of the types of stimuli by looking for the unique stimulus names in this table:

In [ ]:
stimulus_presentations.stimulus_name.unique()

### Plotting one unit's response to drifting gratings

Let's create smaller tables containing only the `drifting_gratings` stimuli, and just the units in V1: 

In [ ]:
stimulus_presentations_dg = stimulus_presentations[stimulus_presentations.stimulus_name == 'drifting_gratings']

units_VISp = units[units.ecephys_structure_acronym == 'VISp']

unit_metrics_VISp = unit_metrics.loc[units_VISp.index.values]

To explore some example drifting grating responses with spikes, we'll select the unit with the highest direction selectivity. When describing gratings stimuli, "direction" refers to the *angle of motion*, which varies between 0 and 360 degrees, while "orientation" refers to the *angle of the grating on individual frames*, which varies between 0 and 180 degrees.

In [ ]:
unit_id = unit_metrics_VISp.g_dsi_dg.idxmax() # index of the row with the maximum value

unit_metrics_VISp.loc[unit_id].g_dsi_dg

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    
    <b>Task:</b> Plot direction selectivity (<tt>g_dsi_dg</tt>) versus orientation selectivity (<tt>g_osi_dg</tt>) for all units in <tt>VISp</tt>.
</div>

In [ ]:
# code goes here

Let's plot this unit's response to all of the different drifting gratings stimulus conditions, which include changes in both. Note that the stimulus presentations table only contains one column to describe the grating angle, which is called `orientation`. We need to pre-process this column a bit to find all of the unique values:

In [ ]:
# use pandas to find the unique orientation values:
unique_orientations = stimulus_presentations_dg.orientation.unique()
print(unique_orientations)

# remove the null value at the end (indicates blank stimulus):
unique_orientations_without_null = unique_orientations[:-1] 
print(unique_orientations_without_null)

# convert to float values so we can sort them:
unique_orientations_as_float = unique_orientations_without_null.astype('float')
print(unique_orientations_as_float)

# perform the sorting:
orientations = np.sort(unique_orientations_as_float)
print(orientations)

Now, we can do the same thing to find the unique temporal frequencies (how fast the grating was moving):

In [ ]:
# use pandas to find the unique temporal frequency values:
unique_temporal_frequencies = stimulus_presentations_dg.temporal_frequency.unique()
print(unique_temporal_frequencies)

# remove the null value at the end (indicates blank stimulus):
unique_temporal_frequencies_without_null = unique_temporal_frequencies[:-1] 
print(unique_temporal_frequencies_without_null)

# convert to float values so we can sort them:
unique_temporal_frequencies_as_float = unique_temporal_frequencies_without_null.astype('float')
print(unique_temporal_frequencies_as_float)

# perform the sorting:
temporal_frequencies = np.sort(unique_temporal_frequencies_as_float)
print(temporal_frequencies)

We can make a raster plot for all presentations of each condition, with each dot representing one spike. Instead of looping through the `spike_times` dictionary like we did previously, we can use a built-in AllenSDK function to greatly speed up the process of finding the relevant spikes. The `presentationwise_spike_times` method makes it easy to extract the spike times for one or more units as a `DataFrame`, given a set of unique presentation IDs (contained in the index column of the `stimulus_presentations` table).

In [ ]:
plt.figure(figsize=(10,5))

for tf_index, tf in enumerate(temporal_frequencies):
    
    for ori_idx, ori in enumerate(orientations):
        
        presentation_ids = stimulus_presentations_dg[
            (stimulus_presentations_dg.temporal_frequency == tf) &
            (stimulus_presentations_dg.orientation == ori)].index.values
        
        df = session.presentationwise_spike_times(presentation_ids, [unit_id])
        
        plt.subplot(len(temporal_frequencies),
                    len(orientations), 
                    ori_idx + tf_index * len(orientations) + 1)
        
        if tf_index == 0:
            plt.title(ori)
            
        if ori_idx == 0:
            plt.ylabel(tf)
        
        if len(df) > 0:
            df['trial_index'] = presentation_ids.searchsorted(df.stimulus_presentation_id.values)
            plt.scatter(df['time_since_stimulus_presentation_onset'], df['trial_index'], s=1, c='k')
            plt.xlim([0, 2]) # each trial is 2 seconds long
            plt.ylim([0,len(presentation_ids)])
            
        if ori_idx == 7 and tf_index == 4:
            plt.xlabel('Time (s)')
            plt.ylabel('Trial')
            
plt.tight_layout()

We can very clearly see this unit's preferred orientation is 225 degrees, and it tends to fire more at lower temporal frequencies.

These raster plots display rich information about a unit's dynamics within a trial. For example, you can see that this unit tends to lock it's firing to a specific phase of the drifting grating at its preferred orientation, which is common for cells in V1.

However, for the purposes of population decoding, we are going to summarize the response to each presentation as a single number, the "firing rate." We can visualize the relative firing rate for each stimulus condition using a "star" plot. This plot was developed specifically for the Allen Brain Observatory datasets, and can be created using a function built in the AllenSDK.

In [ ]:
from allensdk.brain_observatory.ecephys.stimulus_analysis.drifting_gratings import DriftingGratings 
dg = DriftingGratings(session)
dg.make_star_plot(unit_id)

Each presentation is summarized as relative color intensity that's proportional to the firing rate. This is the information we will use for population-level decoding.

Note that this approach rests on two very important assumptions:
1. The decoder knows when the stimulus starts and ends
2. All spikes within this window are weighted equally

These are almost certainly not valid when thinking about how downstream brain regions actually decode activity, but they are very useful for understanding what type of information is represented in whatever brain region we're analyzing. This is a very important point: what we ultimately want to know is "how is information contained in neurons' spiking patterns used to guide behavior?" But, without observing the downstream impact of these spikes, we can only ask, "what information can be decoded from neurons' spiking patterns *in principle*?"

## A faster way to extract firing rates

We will now learn about a shortcut for extracting stimulus-evoked spiking activity, which can easily be applied to many units in parallel. Before we do this, we need to take a brief detour to introduce [Xarray](https://docs.xarray.dev/en/stable/), a Python library for manipulating labeled multi-dimensional arrays.

### Introduction to Xarray

An Xarray `DataArray` is basically a wrapper around a `numpy.ndarray` that includes labels for each axis, so this information is always linked to the underlying data. This means you don't need to use separate variables to keep track of the "meaning" of each axis. You can also think of a `DataArray` as a Pandas `DataFrame` with more than two dimensions (not just rows and columns). Each dimension has its own set of indices, just like the rows of a `DataFrame`.

We chose Xarray `DataArray` as one of the return types for the Neuropixels Visual Coding AllenSDK methods, because this format makes it easy to sub-select data, apply operations over particular axes by name, and better understand what an array actually contains.

Let's take a brief tour of how Xarray is used.

In [ ]:
import xarray as xr # import the xarray library

unit_ids = np.arange(10) # create IDs for 10 units
time_bins = np.arange(0,1,0.01) # times between 0 and 1 s, in 10 ms intervals

firing_rates = np.random.rand(len(unit_ids), len(time_bins))

firing_rates.shape

We now have a matrix of firing rates for 10 units across 100 10-ms bins. However, the meaning of each value in this matrix cannot be understood without looking at the separate arrays, `unit_ids` and `time_bins`. We can use Xarray to link all of this information together:

In [ ]:
da = xr.DataArray(firing_rates, 
                  coords=[unit_ids, time_bins], 
                  dims=["unit_id", "time"])

da

We can now use this `DataArray` to refer to axes by name, rather than by index. For example, averaging over the time axis can be done with the following line:

In [ ]:
da.mean('time')

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    
<b>Task:</b> How would you compute the sum of the firing rates over all units, for each time point?
</div>

In [ ]:
# code goes here

We can also easily select a subset of data for a few units using Xarray's `sel` method:

In [ ]:
da.sel(unit_id=[0,1,2])

And if at any time we need access to the underlying NumPy array that holds the data, we can just call `DataArray.data`:

In [ ]:
type(da.data)

Hopefully this brief demonstration has convinced you that Xarray offers some useful features for analyzing neural data. For a more in-depth introduction, you can read through the [Xarray User Guide](https://docs.xarray.dev/en/stable/user-guide/index.html).

### Computing trial-wise firing rates for one unit

Getting back to the unit we were analyzing previously, we'd like to compute its firing rate for each presentation at a particular temporal frequency (2 Hz). To do this, we'll use an AllenSDK helper method called `presentationwise_spike_counts`. This one is similar to `presentationwise_spike_counts`, except it returns the data as an `xarray.DataArray` with dimensions of `stimulus_presentation_id`, `time_relative_to_stimulus_onset`, and `unit_id`:

In [ ]:
presentations = stimulus_presentations_dg[
            (stimulus_presentations_dg.temporal_frequency == 2.0)]

presentations = presentations.sort_values(by='orientation')

import warnings  
warnings.filterwarnings("ignore") # ignore warnings for the rest of this tutorial
        
da = session.presentationwise_spike_counts(np.arange(0,1.99,0.01),     # time bins
                                           presentations.index.values, # presentation IDs
                                           [unit_id]                   # unit IDs (in this case just one)
                                          )

da

To compute the firing rates, we just need to count spikes by summing across the time dimension, and then divide by two (the total number of seconds in each trial):

In [ ]:
# sum across time dimension
firing_rate_matrix = da.sum('time_relative_to_stimulus_onset') / 2 
print(firing_rate_matrix.shape)

# call np.squeeze to remove the unused unit_id dimension
firing_rates = np.squeeze(firing_rate_matrix) 
print(firing_rates.shape)

plt.bar(np.arange(len(presentations)), firing_rates, width=1)

plt.vlines(np.arange(15, len(presentations), 15), 0, 10, 'k')
plt.ylim([0, 10])
plt.xlabel('Presentation number')
plt.ylabel('Firing rate')
plt.title(f'unit {unit_id}')

Since we sorted our `presentations` table by orientation, trials with the same orientation are grouped together.

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    
<b>Task:</b> Plot the direction tuning curve for this unit by averaging the firing rate over all presentations with the same grating direction.
</div>

In [ ]:
# code goes here

# hint: the first trial for each condition can be found using np.arange(0,len(presentations),15),
# since the trials are already sorted by condition.

We can further condense the information in the firing rate plot by converting firing rates to colors, similarly to what was done in the star plot above:

In [ ]:
plt.figure(figsize=(10,0.25))

plt.imshow(da.sum('time_relative_to_stimulus_onset').T / 2, cmap='RdPu', aspect='auto')
plt.colorbar(label='Firing rate', aspect=1)

## Extracting the population spiking response

We can now use the `presentationwise_spike_counts` method to retrieve the presentation-aligned activity for all the units in V1, just by changing the last input argument:

In [ ]:
unit_ids = unit_metrics_VISp.sort_values(by=['pref_ori_dg']).index.values

da = session.presentationwise_spike_counts(np.arange(0,1.99,0.01), presentations.index.values, unit_ids)

da

Now, the `unit_id` dimension has 94 entries instead of one, but the other dimensions are identical.

Let's plot the normalized firing rates for all units and all presentations. First, we will compute the maximum firing rate for all trials:

In [ ]:
firing_rates = da.sum('time_relative_to_stimulus_onset').T / 2
max_rates = firing_rates.max('unit_id')

Next, we will use a function called `np.tile` to create a matrix that can be used to normalize each trial's firing rates between 0 and 1:

In [ ]:
max_rates_tiled = np.tile(max_rates, (len(unit_ids),1))

norm_firing_rates = firing_rates / max_rates_tiled 

And finally we can plot this normalized firing rate matrix:

In [ ]:
plt.figure(figsize=(10,8))

plt.imshow(norm_firing_rates, cmap='RdPu', aspect='auto', vmax=1)
plt.xlabel('Presentation index')
plt.ylabel('Unit index')
plt.colorbar(label='Firing rate', aspect=20)
_ = plt.ylim([0, len(unit_ids)])

There are a few things worth noting about this plot:

- Because we sorted the `presentations` table by orientation, the transitions between the orientation conditions are obvious for many units
- A large fraction of units are selective to *orientation* but not *direction*, resulting in similar firing rates for orientations separated by 180 degrees

Just from looking at this data, it seems like it will be possible to extract the stimulus orientation from the firing rates within a given trial (the columns in this matrix). We can select one of these columns using the `DataArray.sel` operator:

In [ ]:
one_trial = norm_firing_rates.sel(stimulus_presentation_id = presentations.index.values[0])

plt.figure(figsize=(0.25,10))

# we need to use "expand_dims" because "imshow" expects a 2D array as input:
plt.imshow(np.expand_dims(one_trial, axis=1), cmap='RdPu', aspect='auto', origin='lower')

This represents the normalized firing rate of all units recorded in V1 for a single trial. Let's format the data in such a way that it's possible to feed it into a `scikit-learn` classifier:

In [ ]:
 # retrieve the underlying numpy array and transpose it
responses = norm_firing_rates.data.T

# we're trying to predict the orientation of each trial
labels = presentations.orientation.values.astype('int')

# the conditions are the unique orientations
conditions = np.unique(labels)

print(f'Shape of response matrix: {responses.shape}')
print(f'Shape of labels matrix: {labels.shape}')

The `responses` matrix contains the normalized firing rates for 120 trials and 94 neurons, and the `labels` matrix contains the orientation for 120 trials. Therefore, each trial is represented as a point in a 94-dimensional space. Our goal is to try and draw boundaries in this space that can be used distinguish between the different grating orientations on randomly held out trials.

## Training a classifier on population spiking data

In order to determine how well we can decode the stimulus direction from population activity, we will train a **classifier** on our matrix of firing rates. Whereas regression models try to predict continuous values from the input features, classification models try to predict *labels* (also known as classes) from the input features.

### Support Vector Machines

Let's start with a linear Support Vector Machine (SVM) classifier, which will try to draw linear boundaries between orientation conditions (the labels) in our 94-dimensional firing rate space.

This cartoon shows how we would expect an SVM to behave on a much simpler dataset, which has two dimensions and three conditions:

![SVM illustration](./resources/svm-classifier.png)

SVM computes decision boundaries in feature space that can be used to classify different conditions. If a new data point appears, the SVM classifier will label it based on where it falls with respect to these boundaries.

To train an SVM, we need to import the following methods from `scikit-learn`:

In [ ]:
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

We will first select a random subset of trials for training the classifier:

In [ ]:
total_presentations = responses.shape[0]
num_train = int(total_presentations * 0.75) # Use 75% of trials for training
random_trial_order = np.random.permutation(responses.shape[0])
train_indices = random_trial_order[:num_train]

training_data = responses[train_indices]
training_labels = labels[train_indices]

Next, we'll create the model and fit it to our training data:

In [ ]:
clf = svm.SVC(gamma="scale", kernel="linear")
clf.fit(responses[train_indices], labels[train_indices])

Now that our model has been trained, we can ask it to classify unlabeled data (i.e., the sets of population firing rates that were not included in our original training set):

In [ ]:
test_indices = random_trial_order[num_train:]

test_data = responses[test_indices]
predicted_labels = clf.predict(responses[test_indices])

We can compare the predicted labels to the actual labels in order to assess the classifier's performance:

In [ ]:
actual_labels = labels[test_indices]

accuracy = 1 - (np.count_nonzero(predicted_labels - actual_labels) / predicted_labels.size)

plt.figure(figsize=(5,5))

# add a random offset to make sure the points don't overlap
plt.scatter(actual_labels + np.random.rand(len(test_indices)) * 20, 
            predicted_labels + np.random.rand(len(test_indices)) * 20, 
           s=6)
plt.plot([0,360],[0,360],'--k', alpha=0.1)

plt.title(f'Classification accuracy: {accuracy}')
plt.xlabel('Actual direction (degrees)')
plt.xlabel('Predicted direction (degrees)')

Here, we see that most of the trials are predicted correctly (values on the diagonal line), but it's making some mistakes (off-diagonal points). This indicates that the firing rates of our recorded population do contain information about grating direction, but there are some trials in which the direction is ambiguous, at least to this type of classifier.

We can get a better sense of classification performance by using the `scikit-learn.model_selection.KFold` iterator to automatically split up the data into "train" and "test" sets for 5 iterations. Note that the model is fit independently on each iteration.

In [ ]:
accuracies = []
confusions = []

for train_indices, test_indices in KFold(n_splits=5, shuffle=True).split(responses):
    
    clf = svm.SVC(gamma="scale", kernel="linear")
    clf.fit(responses[train_indices], labels[train_indices])
    
    test_targets = labels[test_indices]
    test_predictions = clf.predict(responses[test_indices])
    
    accuracy = 1 - (np.count_nonzero(test_predictions - test_targets) / test_predictions.size)
    print(accuracy)
    
    accuracies.append(accuracy)
    confusions.append(confusion_matrix(y_true=test_targets, y_pred=test_predictions, labels=conditions))
    
print(f"\nmean accuracy: {np.mean(accuracies)}")
print(f"chance: {1/conditions.size}")

Our SVM model performs much better than chance levels, for which we'd expect a classification accuracy of 12.5%.

We can visualize its performance in more detail using a confusion metrix. In the ideal confusion matrix, we would want to have maximum values along the diagonal, and zeros at all other points. Any off-diagonal points that are non-zero show which conditions our classifier is "confusing" with one another.

In [ ]:
def plot_confusion_matrix(confusions):
    
    plt.figure(figsize=(6,6))
    
    mean_confusion = np.mean(confusions, axis=0)

    plt.imshow(mean_confusion)
    plt.colorbar()

    plt.xlabel("actual")
    plt.ylabel("predicted")
    
plot_confusion_matrix(confusions)

When the model makes mistakes, it tends to mis-classify trials by 180 degrees, which is expected given the fact that many units are selective to orientation but not direction.

### Random Forest

To try to avoid some of these errors, we can try a nonlinear classifier (Random Forest) which uses a set of decision trees to determine the label for each trial.

This cartoon shows how a Random Forest behaves on the much simpler case we saw previously:

![Random forest illustration](./resources/random-forest-classifier.png)

Instead of trying to draw explicit boundaries in feature space, this method creates a set of "decision trees" that can be used to classify individual points. In this simple example, only two decisions are needed to classify all points correctly. However, if the feature space is more complicated, these decision trees can grow much larger. To prevent the classification from landing in a local minimum, the Random Forest creates multiple decision trees (hence the name "forest"), and uses majority voting to select the final labels. Random Forest classifiers are highly adaptable, since they make few assumptions about the underlying structure of the data.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

accuracies = []
confusions = []

for train_indices, test_indices in KFold(n_splits=5, shuffle=True).split(responses):
    
    clf = RandomForestClassifier()
    clf.fit(responses[train_indices], labels[train_indices])
    
    test_targets = labels[test_indices]
    test_predictions = clf.predict(responses[test_indices])
    
    accuracy = 1 - (np.count_nonzero(test_predictions - test_targets) / test_predictions.size)
    print(accuracy)
    
    accuracies.append(accuracy)
    confusions.append(confusion_matrix(y_true=test_targets, y_pred=test_predictions, labels=conditions))
    
print(f"\nmean accuracy: {np.mean(accuracies)}")
print(f"chance: {1/conditions.size}")

This model appears to be doing even better than the SVM.

In [ ]:
plot_confusion_matrix(confusions)

## Relationship between population size and decoding accuracy

As a final exercise in this tutorial, we'll examine how the size of the simultaneously recorded population affects decoding accuracy. In any physiology experiment, we only have a very small window into the overall population response. For example, there are about 500,000 neurons in mouse V1, so in this case we are measuring around 0.02% of the firing rates in this region.

As the number of simultaneously recorded neurons increases, we expect that our ability to decode stimulus identity will improve. Let's try to get a sense for how this changes with the number of neurons we use to train the classifier:

In [ ]:
total_neurons = np.arange(5, responses.shape[1], 5)

num_iterations = 10
num_splits = 5
accuracies = np.zeros((len(total_neurons), num_iterations, num_splits))

for ind1, N in enumerate(total_neurons):
    
    print(f'total neurons: {N}')
    
    for ind2, iteration in enumerate(range(0,num_iterations)):
        
        random_neuron_order = np.random.permutation(responses.shape[1])
        selected_neurons = random_neuron_order[:N]
        responses_subset = responses[:,selected_neurons]
        
        ind3 = 0

        for train_indices, test_indices in KFold(n_splits=num_splits, shuffle=True).split(responses_subset):

            clf = RandomForestClassifier()
            clf.fit(responses_subset[train_indices], labels[train_indices])

            test_targets = labels[test_indices]
            test_predictions = clf.predict(responses_subset[test_indices])

            accuracy = 1 - (np.count_nonzero(test_predictions - test_targets) / test_predictions.size)
            accuracies[ind1, ind2, ind3] = accuracy
            
            ind3 += 1

We can visualize the relationship between neuron count and classification accuracy by averaging over the training splits and iterations:

In [ ]:
plt.plot(total_neurons, np.mean(np.mean(accuracies,2),1))
plt.xlabel('Total neurons')
plt.ylabel('Mean classification accuracy')

Even with only 5 neurons, the random forest classifier still performs above chance (0.125). Performance saturates after around 60 neurons are included in this analysis.

In this tutorial, we learned how to decode drifting grating orientations from population firing rates in V1. Using both a linear (SVM) and nonlinear (Random Forest) classifer, we were able to achieve high levels of decoding accuracy. This only scratches the surface of population-level decoding, and raises a number of follow-up questions that can be pursued as problem sets.

In the next tutorial, we will examine how between-neuron correlations affect decoding.